In [1]:
from ultralytics import YOLO
from pathlib import Path

# пути
DATA_ROOT = Path(r"datasets/sirius/tlogo_sirius")
DATA_YAML = DATA_ROOT / "data.yaml"


model = YOLO("yolo12s.pt")

EPOCHS = 50
IMGSZ  = 840   # Максимальный размер в валидационной выборке, даже если изображение сожмется - в целом не страшно
BATCH  = 0.75                                           # Занимаем 75% видеопамяти

train_args = dict(
    data=str(DATA_YAML),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0,
    workers=4,
    amp=True,                                           # вообще модель s можно и без AMP обучать, но времени у меня мало...
    single_cls=True,
    
    hsv_h=0.0, hsv_s=0.3, hsv_v=0.3,   # — уменьшаем стандартные параметры YOLO, дабы не сильно зашумлять данные (в исходном наборе шума почти нет)
    degrees=5.0, translate=0.05, scale=0.10, shear=2.0, # имитируем съемку с разных ракурсов и расстояний
    perspective=0.0, flipud=0.0, fliplr=0.5,            # Перспектива (3D) тут не нужна, а вот повороты весьма кстати
    mosaic=0.0, mixup=0.0,
    cos_lr=True,                                        # должно улучшить сходимость
    lr0=0.003, lrf=0.01,                                # чуть ускорим сходимость в начале обучения
    momentum=0.937, weight_decay=0.0005,
    patience=20,
    cache=True,
    verbose=True,
)

results = model.train(**train_args)

print("Best weights:", model.trainer.best)


New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=0.75, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=datasets\sirius\tlogo_sirius\data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.3, hsv_v=0.3, imgsz=840, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=train7, nbs=64, n

C:\Users\mrsli\anaconda3\envs\work_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=864 at 75.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4070) 11.99G total, 0.13G reserved, 0.10G allocated, 11.76G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     9253523       39.22         1.539            43          2922        (1, 3, 864, 864)                    list
     9253523       78.45         2.603            51          2989        (2, 3, 864, 864)                    list
     9253523       156.9         4.467            69          3181        (4, 3, 864, 864)                    list
     9253523       313.8         8.319         136.2          3248        (8, 3, 864, 864)                    list
     9253523       6

In [11]:
model = YOLO(r"runs\detect\train7\weights\best.pt")

metrics = model.val(data=r"datasets\sirius\tlogo_sirius\data.yaml", device=0, batch=1, imgsz=1280)
# усреднённые по классам метрики 
P = float(metrics.box.mp)
R = float(metrics.box.mr)
F1 = 2 * P * R / (P + R + 1e-16)

print(f"Precision: {P:.4f}")
print(f"Recall:    {R:.4f}")
print(f"F1-score:  {F1:.4f}")

Ultralytics 8.3.199  Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLOv12s summary (fused): 159 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 135.684.8 MB/s, size: 47.9 KB)
val: Scanning C:\Anaconda_notebooks\datasets\sirius\tlogo_sirius\valid2\labels.cache... 146 images, 42 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 146/146 146.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 146/146 47.1it/s 3.1s0.1s
                   all        146        120      0.913      0.708      0.779      0.557
Speed: 0.6ms preprocess, 17.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to C:\Anaconda_notebooks\runs\detect\val37
Precision: 0.9131
Recall:    0.7083
F1-score:  0.7978
